In [18]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [19]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [20]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [48]:
mat_file_list=['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satimage-2.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']

In [49]:
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

In [50]:
data=loadmat("C:/Users/harish/Downloads/data/cardio.mat")
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [51]:
len(data)

5

In [52]:

df_columns=['Data','#Sample','#Dimensions','Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS','ABOD','IFOREST','FEATUREBAGGING']

In [53]:
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


In [54]:
from time import time
random_state = np.random.RandomState(42)

for mat_file in mat_file_list:
   print("\n... Processing", mat_file, '...')
   mat = loadmat(os.path.join("C:/Users/harish/Downloads/data", mat_file))

   X = mat['X']
   y = mat['y'].ravel()
   outliers_fraction = np.count_nonzero(y) / len(y)
   outliers_percentage = round(outliers_fraction * 100, ndigits=4)

   # construct containers for saving results
   roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
   prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
   time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

   # 60% data for training and 40% for testing
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                       random_state=random_state)

   # standardizing data for processing
   X_train_norm, X_test_norm = standardizer(X_train, X_test)

   classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
       contamination=outliers_fraction),
       'Cluster-based Local Outlier Factor': CBLOF(
           contamination=outliers_fraction, check_estimator=False,
           random_state=random_state),
       'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                         random_state=random_state),
       'Histogram-base Outlier Detection (HBOS)': HBOS(
           contamination=outliers_fraction),
       'Isolation Forest': IForest(contamination=outliers_fraction,
                                   random_state=random_state),
       'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
       'Local Outlier Factor (LOF)': LOF(
           contamination=outliers_fraction),
       'Minimum Covariance Determinant (MCD)': MCD(
           contamination=outliers_fraction, random_state=random_state),
       'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
       'Principal Component Analysis (PCA)': PCA(
           contamination=outliers_fraction, random_state=random_state),
   }

   for clf_name, clf in classifiers.items():
       t0 = time()
       clf.fit(X_train_norm)
       test_scores = clf.decision_function(X_test_norm)
       t1 = time()
       duration = round(t1 - t0, ndigits=4)
       time_list.append(duration)

       roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
       prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

       print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
             'execution time: {duration}s'.format(
           clf_name=clf_name, roc=roc, prn=prn, duration=duration))

       roc_list.append(roc)
       prn_list.append(prn)

   temp_df = pd.DataFrame(time_list).transpose()
   temp_df.columns = df_columns
   time_df = pd.concat([time_df, temp_df], axis=0)

   temp_df = pd.DataFrame(roc_list).transpose()
   temp_df.columns = df_columns
   roc_df = pd.concat([roc_df, temp_df], axis=0)

   temp_df = pd.DataFrame(prn_list).transpose()
   temp_df.columns = df_columns
   prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571, execution time: 0.2808s
Cluster-based Local Outlier Factor ROC:0.7684, precision @ rank n:0.4643, execution time: 0.3278s
Feature Bagging ROC:0.7799, precision @ rank n:0.5, execution time: 1.0s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714, execution time: 0.107s
Isolation Forest ROC:0.8527, precision @ rank n:0.5714, execution time: 0.8316s
K Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5, execution time: 0.1369s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643, execution time: 0.1249s


C:\Users\harish\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286, execution time: 1.1433s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5, execution time: 0.061s
Principal Component Analysis (PCA) ROC:0.7997, precision @ rank n:0.5, execution time: 0.0859s

... Processing cardio.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.5763, precision @ rank n:0.1875, execution time: 0.9165s
Cluster-based Local Outlier Factor ROC:0.8221, precision @ rank n:0.4844, execution time: 0.3378s
Feature Bagging ROC:0.4879, precision @ rank n:0.1406, execution time: 1.2963s
Histogram-base Outlier Detection (HBOS) ROC:0.8453, precision @ rank n:0.4688, execution time: 0.012s
Isolation Forest ROC:0.9414, precision @ rank n:0.5, execution time: 0.7211s
K Nearest Neighbors (KNN) ROC:0.6959, precision @ rank n:0.2812, execution time: 0.2529s
Local Outlier Factor (LOF) ROC:0.4715, precision @ rank n:0.125, execution time: 0.1449s


C:\Users\harish\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Users\harish\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-152.076751858648862 > -250.756822706501453). You may want to try with a higher value of support_fraction (current value: 0.511).
  RuntimeWarning)


Minimum Covariance Determinant (MCD) ROC:0.8778, precision @ rank n:0.3906, execution time: 1.0294s
One-class SVM (OCSVM) ROC:0.9507, precision @ rank n:0.5938, execution time: 0.1109s
Principal Component Analysis (PCA) ROC:0.9638, precision @ rank n:0.6875, execution time: 0.009s

... Processing glass.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7104, precision @ rank n:0.25, execution time: 0.1269s
Cluster-based Local Outlier Factor ROC:0.8506, precision @ rank n:0.25, execution time: 0.077s
Feature Bagging ROC:0.7043, precision @ rank n:0.25, execution time: 0.1039s
Histogram-base Outlier Detection (HBOS) ROC:0.6524, precision @ rank n:0.0, execution time: 0.005s
Isolation Forest ROC:0.7195, precision @ rank n:0.25, execution time: 0.5657s
K Nearest Neighbors (KNN) ROC:0.7805, precision @ rank n:0.25, execution time: 0.016s
Local Outlier Factor (LOF) ROC:0.7774, precision @ rank n:0.25, execution time: 0.004s
Minimum Covariance Determinant (MCD) ROC:0.7165, precision @ rank n:0

...

Minimum Covariance Determinant (MCD) ROC:0.8102, precision @ rank n:0.157, execution time: 5.221s
One-class SVM (OCSVM) ROC:0.8456, precision @ rank n:0.3857, execution time: 6.3054s
Principal Component Analysis (PCA) ROC:0.8459, precision @ rank n:0.3857, execution time: 0.2419s

... Processing musk.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.1172, precision @ rank n:0.0526, execution time: 3.471s
Cluster-based Local Outlier Factor ROC:1.0, precision @ rank n:1.0, execution time: 0.7036s
Feature Bagging ROC:0.5741, precision @ rank n:0.3421, execution time: 17.527s
Histogram-base Outlier Detection (HBOS) ROC:1.0, precision @ rank n:1.0, execution time: 0.0959s
Isolation Forest ROC:1.0, precision @ rank n:1.0, execution time: 2.2597s
K Nearest Neighbors (KNN) ROC:0.8305, precision @ rank n:0.2895, execution time: 3.1102s
Local Outlier Factor (LOF) ROC:0.5682, precision @ rank n:0.2368, execution time: 2.5306s
Minimum Covariance Determinant (MCD) ROC:1.0, precision @ rank n:1.0, e

C:\Users\harish\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.3402, precision @ rank n:0.0, execution time: 2.1887s
One-class SVM (OCSVM) ROC:0.4515, precision @ rank n:0.0, execution time: 1.9509s
Principal Component Analysis (PCA) ROC:0.4682, precision @ rank n:0.0, execution time: 0.075s

... Processing pendigits.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.6858, precision @ rank n:0.1408, execution time: 3.1502s
Cluster-based Local Outlier Factor ROC:0.9661, precision @ rank n:0.3662, execution time: 0.3828s
Feature Bagging ROC:0.4213, precision @ rank n:0.0986, execution time: 5.9696s
Histogram-base Outlier Detection (HBOS) ROC:0.9224, precision @ rank n:0.2817, execution time: 0.018s
Isolation Forest ROC:0.9387, precision @ rank n:0.2535, execution time: 1.2366s
K Nearest Neighbors (KNN) ROC:0.77, precision @ rank n:0.1268, execution time: 0.9684s
Local Outlier Factor (LOF) ROC:0.3885, precision @ rank n:0.1127, execution time: 0.8685s
Minimum Covariance Determinant (MCD) ROC:0.816, precision 

C:\Users\harish\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.810717311975694 > -74.935231523318947). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\Users\harish\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.076223496828817 > -79.030793026071876). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\Users\harish\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.764767161364979 > -77.107451487662928). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C

Minimum Covariance Determinant (MCD) ROC:0.99, precision @ rank n:0.7473, execution time: 18.7593s
One-class SVM (OCSVM) ROC:0.9914, precision @ rank n:0.9532, execution time: 76.6382s
Principal Component Analysis (PCA) ROC:0.9893, precision @ rank n:0.9482, execution time: 0.059s

... Processing vertebral.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.331, precision @ rank n:0.0, execution time: 0.0969s
Cluster-based Local Outlier Factor ROC:0.4399, precision @ rank n:0.0, execution time: 0.1199s
Feature Bagging ROC:0.4207, precision @ rank n:0.0714, execution time: 0.1131s
Histogram-base Outlier Detection (HBOS) ROC:0.2483, precision @ rank n:0.0, execution time: 0.003s
Isolation Forest ROC:0.3632, precision @ rank n:0.0714, execution time: 0.5547s
K Nearest Neighbors (KNN) ROC:0.3319, precision @ rank n:0.0, execution time: 0.02s
Local Outlier Factor (LOF) ROC:0.4042, precision @ rank n:0.0, execution time: 0.01s
Minimum Covariance Determinant (MCD) ROC:0.4042, precision @ rank n

In [55]:
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,mnist,7603,100,9.2069,0.7925,0.8498,0.7321,0.5769,0.7851,0.8556,0.742,0.8102,0.8456,0.8459
0,musk,3062,166,3.1679,0.1172,1,0.5741,1,1,0.8305,0.5682,1,1,1
0,optdigits,5216,64,2.8758,0.4708,0.7832,0.4193,0.8273,0.7225,0.369,0.4201,0.3402,0.4515,0.4682
0,pendigits,6870,16,2.2707,0.6858,0.9661,0.4213,0.9224,0.9387,0.77,0.3885,0.816,0.9371,0.9313
0,pima,768,8,34.8958,0.7078,0.7076,0.6468,0.7375,0.7115,0.7336,0.6545,0.7155,0.666,0.6815
0,satimage-2,5803,36,1.2235,0.7862,0.9967,0.4706,0.9561,0.9878,0.9434,0.4728,0.9943,0.9919,0.9574


In [56]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,mnist,7603,100,9.2069,0.3549,0.4232,0.3549,0.1263,0.3038,0.4198,0.3584,0.157,0.3857,0.3857
0,musk,3062,166,3.1679,0.0526,1,0.3421,1,1,0.2895,0.2368,1,1,0.9737
0,optdigits,5216,64,2.8758,0.0169,0,0.0339,0.2203,0,0,0.0339,0,0,0
0,pendigits,6870,16,2.2707,0.1408,0.3662,0.0986,0.2817,0.2535,0.1268,0.1127,0.0986,0.3239,0.3239
0,pima,768,8,34.8958,0.5421,0.514,0.4766,0.5701,0.5888,0.5888,0.4953,0.5421,0.5234,0.5514
0,satimage-2,5803,36,1.2235,0.25,0.9286,0.0357,0.6429,0.8571,0.3571,0,0.5714,0.8929,0.7857


In [57]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.2808,0.3278,1,0.107,0.8316,0.1369,0.1249,1.1433,0.061,0.0859
0,cardio,1831,21,9.6122,0.9165,0.3378,1.2963,0.012,0.7211,0.2529,0.1449,1.0294,0.1109,0.009
0,glass,214,9,4.2056,0.1269,0.077,0.1039,0.005,0.5657,0.016,0.004,0.061,0.002,0.003
0,ionosphere,351,33,35.8974,0.2169,0.083,0.1709,0.02,0.7107,0.039,0.011,0.1359,0.009,0.005
0,mnist,7603,100,9.2069,10.9278,2.0198,62.1834,0.079,3.532,10.0063,9.4416,5.221,6.3054,0.2419
0,musk,3062,166,3.1679,3.471,0.7036,17.527,0.0959,2.2597,3.1102,2.5306,19.5098,1.5401,0.2479
0,optdigits,5216,64,2.8758,4.1636,0.8395,24.568,0.051,1.7065,3.648,2.9124,2.1887,1.9509,0.075
0,pendigits,6870,16,2.2707,3.1502,0.3828,5.9696,0.018,1.2366,0.9684,0.8685,4.0797,1.4592,0.012
0,pima,768,8,34.8958,0.2449,0.1399,0.1549,0.006,0.6576,0.059,0.017,0.1219,0.018,0.005
0,satimage-2,5803,36,1.2235,3.0173,0.7786,9.3155,0.027,1.6526,1.3932,1.1573,4.5684,1.5631,0.037
